In [1]:
from http import HTTPStatus
from pathlib import Path
import os
from os.path import exists
import requests
import time

# where 01 to 56 are each state/territory:
#  * populations - http://censusdata.ire.org/01/all_140_in_01.P1.csv
#  * centroids - https://www2.census.gov/geo/docs/reference/cenpop2020/tract/CenPop2020_Mean_TR01.txt
parent_directory = Path(os.path.abspath("")).parent / "data"


def save_download_data_to_path(file_path_to_save: str, url_to_download: str):
    try:
        response = requests.get(url_to_download, stream=True)
        if response.status_code == HTTPStatus.OK and "404: Page Not Found" not in str(response.content):
            with open(file_path_to_save, "wb") as file_to_write:
                file_to_write.write(response.content)
        else:
            print(f"Skipping {url_to_download} because it was missing or not found!")
    except Exception as e:
        print("Unable to retrieve data for some reason", e)


def download_and_save_file_if_needed(url_to_download: str, file_path_to_save: str):
    if not exists(file_path_to_save):
        print(f"Downloading file as it does not exist: {url_to_download}")
        save_download_data_to_path(str(file_path_to_save), url_to_download)
        time.sleep(1)


# data directory for stored population data
populations_centroids_data_directory = parent_directory / "populations_tracts"
for idx in range(1, 57):
    state_code = str(idx).zfill(2)
    file_to_download = f"all_140_in_{state_code}.P1.csv"
    url_to_download = f"http://censusdata.ire.org/{state_code}/{file_to_download}"
    file_path_to_save = Path(populations_centroids_data_directory) / file_to_download
    download_and_save_file_if_needed(url_to_download, file_path_to_save)
print("Completed downloading population tract data")


# data directory for stored centroid data
tracts_centroids_data_directory = parent_directory / "centroids_tracts"
for idx in range(1, 57):
    state_code = str(idx).zfill(2)
    file_to_download = f"CenPop2020_Mean_TR{state_code}.txt"
    url_to_download = f"https://www2.census.gov/geo/docs/reference/cenpop2020/tract/{file_to_download}"
    file_path_to_save = Path(tracts_centroids_data_directory) / file_to_download
    download_and_save_file_if_needed(url_to_download, file_path_to_save)
print("Completed downloading centroid tract data")

Skipping http://censusdata.ire.org/03/all_140_in_03.P1.csv because it was missing or not found!
Skipping http://censusdata.ire.org/07/all_140_in_07.P1.csv because it was missing or not found!
Skipping http://censusdata.ire.org/14/all_140_in_14.P1.csv because it was missing or not found!
Skipping http://censusdata.ire.org/43/all_140_in_43.P1.csv because it was missing or not found!
Skipping http://censusdata.ire.org/52/all_140_in_52.P1.csv because it was missing or not found!
Completed downloading population tract data
Skipping https://www2.census.gov/geo/docs/reference/cenpop2020/tract/CenPop2020_Mean_TR03.txt because it was missing or not found!
Skipping https://www2.census.gov/geo/docs/reference/cenpop2020/tract/CenPop2020_Mean_TR07.txt because it was missing or not found!
Skipping https://www2.census.gov/geo/docs/reference/cenpop2020/tract/CenPop2020_Mean_TR14.txt because it was missing or not found!
Skipping https://www2.census.gov/geo/docs/reference/cenpop2020/tract/CenPop2020_Mea

In [2]:
# consolidate data into pandas data frames
import pandas as pd

# population data
populations_centroids_csv_files = [
    f"{populations_centroids_data_directory}/all_140_in_{str(idx).zfill(2)}.P1.csv"
    for idx in range(1, 57)
    if exists(f"{populations_centroids_data_directory}/all_140_in_{str(idx).zfill(2)}.P1.csv")
]
populations_df = pd.concat(map(pd.read_csv, populations_centroids_csv_files), ignore_index=True)

# tracts centroids data
tracts_centroids_csv_files = [
    f"{tracts_centroids_data_directory}/CenPop2020_Mean_TR{str(idx).zfill(2)}.txt"
    for idx in range(1, 57)
    if exists(f"{tracts_centroids_data_directory}/CenPop2020_Mean_TR{str(idx).zfill(2)}.txt")
]
centroids_df = pd.concat(map(pd.read_csv, tracts_centroids_csv_files), ignore_index=True)

# populations df 'GEOID' column corresponds to STATEFP COUNTYFP TRACTCE
centroids_df["GEOID"] = (
    centroids_df.STATEFP.astype(str) + centroids_df.COUNTYFP.astype(str).str.zfill(3) + centroids_df.TRACTCE.astype(str)
).astype(int)

unified_df = pd.merge(populations_df, centroids_df, on="GEOID")
unified_df

,GEOID,SUMLEV,STATE,COUNTY,CBSA,CSA,NECTA,CNECTA,NAME,POP100,...,POP100.2000,HU100.2000,P001001,P001001.2000,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE
0,1003990000,140,1,3,19300,380,NaN,NaN,Census Tract 9900,0,...,0,0,0,0,1,3,990000,0,30.328800,-87.882973
1,1005950100,140,1,5,21640,999,NaN,NaN,Census Tract 9501,3321,...,3840,1725,3321,3840,1,5,950100,3126,31.968728,-85.160270
2,1005950200,140,1,5,21640,999,NaN,NaN,Census Tract 9502,4264,...,4342,1464,4264,4342,1,5,950200,3363,31.885436,-85.470069
3,1005950300,140,1,5,21640,999,NaN,NaN,Census Tract 9503,1638,...,2152,922,1638,2152,1,5,950300,1370,31.788329,-85.558806
4,1005950400,140,1,5,21640,999,NaN,NaN,Census Tract 9504,4303,...,7915,3199,4303,7915,1,5,950400,3998,31.686535,-85.589767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26780,56023978400,140,56,23,99999,999,NaN,NaN,Census Tract 9784,3534,...,3474,1585,3534,3474,56,23,978400,3042,41.788941,-110.544129
26781,56037970602,140,56,37,40540,999,NaN,NaN,Census Tract 9706.02,3471,...,3429,1100,3471,3429,56,37,970602,3247,41.497553,-109.456305
26782,56037970700,140,56,37,40540,999,NaN,NaN,Census Tract 9707,3709,...,3599,1416,3709,3599,56,37,970700,3816,41.493618,-109.489711
26783,56045951100,140,56,45,99999,999,NaN,NaN,Census Tract 9511,3314,...,2985,1494,3314,2985,56,45,951100,3336,43.969553,-104.406191


In [3]:
# convert regular data frame into pandas dataframe - https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html
import geog
import geojson
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, Point
import reverse_geocode
from data.locations import raw_locations


RADIUS_TO_SEARCH_MI = (100 * 1000) / 0.621371  # radius to search in miles in km
POINTS_FOR_CIRCLE = 300

unified_gdf = gpd.GeoDataFrame(unified_df, geometry=gpd.points_from_xy(unified_df.LONGITUDE, unified_df.LATITUDE))

# generate circle from point and radius with angles - ex - https://gis.stackexchange.com/a/268251
angles = np.linspace(0, 360, POINTS_FOR_CIRCLE)

# - get list of census tracts in radius - https://gis.stackexchange.com/a/67872
# - cross reference census tracts and query their populations and store it for each point
def get_population_for_location(location):
    polygon_data = Polygon(geog.propagate(Point(location.coordinates), angles, RADIUS_TO_SEARCH_MI))
    polygon = gpd.GeoDataFrame({"overlapping_area": [None]}, geometry=[polygon_data])
    return gpd.overlay(unified_gdf, polygon, how="intersection")["POPULATION"].sum()


weighted_location_data = []
for location in raw_locations:
    weighted_location_data.append(
        {"location": location, "population": get_population_for_location(geojson.Point((location[0], location[1])))}
    )

pd.DataFrame.from_dict(weighted_location_data)

/Users/madhu/PycharmProjects/aptible_geodata/.venv/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


,location,population
0,"[-77.1528, 39.085]",7559490
1,"[-74.6551, 40.3431]",9002634
2,"[-74.6551, 40.3431]",9002634
3,"[-71.0589, 42.3601]",6649667
4,"[-111.891, 40.7608]",1434799
5,"[-121.6784, 45.5152]",204694
6,"[-74.006, 40.7128]",10340006
7,"[-94.2102, 36.3724]",509743
8,"[-73.5673, 45.5017]",221109
9,"[-87.6298, 41.8781]",8308029


In [4]:
import geojson

total_latitude_weighted_with_population = 0
total_longitude_weighted_with_population = 0
total_population_in_radius = 0

for weighted_location in weighted_location_data:
    # - sum those populations and multiply that by lat, long
    total_population_in_radius += weighted_location["population"]
    total_latitude_weighted_with_population += weighted_location["location"][0] * weighted_location["population"]
    total_longitude_weighted_with_population += weighted_location["location"][1] * weighted_location["population"]

# - divide by total number of points * total population for weighted lat/long
approximate_weighted_center_lat = total_latitude_weighted_with_population / total_population_in_radius
approximate_weighted_center_long = total_longitude_weighted_with_population / total_population_in_radius
approximate_weighted_center = geojson.Point((approximate_weighted_center_lat, approximate_weighted_center_long))

reverse_geocode.search([(approximate_weighted_center_long, approximate_weighted_center_lat)])

[{'country_code': 'US', 'city': 'Patton', 'country': 'United States'}]

In [5]:
from keplergl import KeplerGl
import geojson
import json

map_1 = KeplerGl(
    config={
        "version": "v1",
        "config": {
            "mapState": {
                "bearing": 0,
                "dragRotate": True,
                "latitude": 39.50,
                "longitude": -98.35,
                "pitch": 0,
                "zoom": 2.75,
                "isSplit": False,
            },
        },
    }
)

locations = geojson.GeometryCollection(
    [geojson.Point((raw_location[0], raw_location[1])) for raw_location in raw_locations]
)

map_1.add_data(data=json.dumps(locations), name="employee locations")
map_1.add_data(data=json.dumps(approximate_weighted_center), name="center")
map_1.save_to_html(file_name='../outputs/2-population-weighted-centroid.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to ../outputs/2-population-weighted-centroid.html!


In [6]:
%%html
<iframe width="700" height="500" src="../outputs/2-population-weighted-centroid.html" frameborder="0" allowfullscreen></iframe>